In [9]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [2]:
raw_dataset = load_dataset('kde4', lang1='en', lang2='fr')
raw_dataset

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [3]:
split_dataset = raw_dataset['train'].train_test_split(train_size=.9, seed=20)
split_dataset['validation'] = split_dataset.pop('test')
split_dataset["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [6]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline('translation', model=model_checkpoint)

[{'translation_text': 'Mon nom est Tyler'}]

In [7]:
translator('my name is tyler')

[{'translation_text': 'Mon nom est Tyler'}]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [14]:
en_inputs = split_dataset['train'][1]['translation']['en']
fr_inputs = split_dataset['train'][1]['translation']['fr']

inputs = tokenizer(en_inputs, text_target=fr_inputs)
inputs   # the input here is english tokens, and the label is the french translation

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [20]:
max_length = 128

def preprocess(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs


In [16]:
tokenized_datasets = split_dataset.map(preprocess, batched=True, remove_columns=split_dataset['train'].column_names)

Map:   0%|                                    | 0/189155 [00:00<?, ? examples/s]


TypeError: string indices must be integers